In [1]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

2022-04-27 14:38:45.903556: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 14:38:45.903586: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
def plot_input_img(i):
    plt.imshow(X_train[i], cmap='binary')
    plt.title(y_train[i])
    plt.show()

In [4]:
# Pre Process the images

# Normalizing the image to [0,1] range
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255

# Reshape / expand the dimentions of images to (28, 28, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# convert classes to one hot vectors
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)

In [5]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = (28, 28, 1), activation= 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64, (3,3), activation= 'relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

2022-04-27 14:39:17.842633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-27 14:39:17.842710: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-27 14:39:17.842743: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2022-04-27 14:39:17.844463: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [7]:
model.compile(optimizer='adam', loss= keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [8]:
# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

#Earlystopping
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=4, verbose=1)

# Model check point
mc = ModelCheckpoint("./bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only=True)

cb = [es, mc]

In [9]:
his = model.fit(X_train, y_train, epochs=5, validation_split=0.3, callbacks=cb)

2022-04-27 14:39:33.851222: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 131712000 exceeds 10% of free system memory.


Epoch 1/5
   3/1313 [..............................] - ETA: 1:19 - loss: 2.3199 - accuracy: 0.1146

2022-04-27 14:39:36.055473: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-04-27 14:39:36.055589: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-04-27 14:39:36.131335: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.
2022-04-27 14:39:36.133829: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 17981568 exceeds 10% of free system memory.


1311/1313 [============================>.] - ETA: 0s - loss: 0.2182 - accuracy: 0.9335
Epoch 1: val_accuracy improved from -inf to 0.97128, saving model to ./bestmodel.h5
1313/1313 [==============================] - 40s 29ms/step - loss: 0.2181 - accuracy: 0.9335 - val_loss: 0.0908 - val_accuracy: 0.9713
Epoch 2/5
1313/1313 [==============================] - ETA: 0s - loss: 0.0748 - accuracy: 0.9768
Epoch 2: val_accuracy improved from 0.97128 to 0.98394, saving model to ./bestmodel.h5
1313/1313 [==============================] - 40s 31ms/step - loss: 0.0748 - accuracy: 0.9768 - val_loss: 0.0566 - val_accuracy: 0.9839
Epoch 3/5
1312/1313 [============================>.] - ETA: 0s - loss: 0.0554 - accuracy: 0.9836
Epoch 3: val_accuracy did not improve from 0.98394
1313/1313 [==============================] - 44s 33ms/step - loss: 0.0555 - accuracy: 0.9836 - val_loss: 0.0536 - val_accuracy: 0.9836
Epoch 4/5
1311/1313 [============================>.] - ETA: 0s - loss: 0.0450 - accuracy: 0.

In [10]:
model_S = keras.models.load_model("/home/EAlderson/project/hdr/bestmodel.h5")

In [11]:
score = model_S.evaluate(X_test, y_test)

print(f"The model Accuracy is {score[1]}")

313/313 [==============================] - 4s 13ms/step - loss: 0.0335 - accuracy: 0.9891
The model Accuracy is 0.9890999794006348
